In [ ]:
import numpy as np
import tensorflow as tf

import os
import json
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import cv2
from utils_train.Datagenerator import LabelEncoder

In [ ]:
min_iou = 0.7
w_gt = tf.constant([1,2,3,4], tf.float32)
h_gt = tf.constant([1,2,3,4], tf.float32)


####distance_detection_offset
a = 1
b = -(h_gt+w_gt)
c=w_gt * h_gt * ((1 - min_iou) / (1 + min_iou))

discriminant = tf.sqrt(b ** 2 - 4 * a * c)
distance_detection_offset = (-b + discriminant) / (2.0)
####distance_detection_offset

####distance_detection_in_gt
a = 4
b = -2 * (h_gt + w_gt)
c = (1 - min_iou) * w_gt * h_gt

discriminant = tf.sqrt(b ** 2 - 4 * a * c)
distance_detection_in_gt = (-b + discriminant) / (2.0)
####distance_detection_in_gt

####distance_gt_in_detection
a = 4 * min_iou
b = (2 * min_iou) * (w_gt + h_gt)
c = (min_iou - 1) * w_gt * h_gt

discriminant = tf.sqrt(b ** 2 - 4 * a * c)
distance_gt_in_detection = (-b + discriminant) / (2.0)
####distance_gt_in_detection

ans = tf.reduce_min([distance_detection_offset,
                        distance_gt_in_detection,
                        distance_detection_in_gt], axis=0)

In [ ]:
[_tfrecords], dataset_info = tfds.load(name="coco/2017", split=["train"], with_info=True, shuffle_files=True)

In [ ]:
def get_pred_wh(shape):
    h, w = shape
    base_step = 1
    shifts_x = np.arange(0, (w - 1) * base_step + 1, base_step, dtype=np.float32)
    shifts_y = np.arange(0, (h - 1) * base_step + 1, base_step, dtype=np.float32)
    shift_y, shift_x = np.meshgrid(shifts_y, shifts_x)
    base_loc = np.stack((shift_x, shift_y), axis=0)
    return base_loc

def gaussian2D(shape, sigma_x=1, sigma_y=1):
    m, n = [(ss - 1.0) / 2.0 for ss in shape]
    y, x = np.ogrid[-m : m + 1, -n : n + 1]

    # h = np.exp(-(x * x + y * y) / (2 * sigma * sigma))
    h = np.exp(-(x * x / (2 * sigma_x * sigma_x) + y * y / (2 * sigma_y * sigma_y)))
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    return h

def bbox_areas_log_np(bbox):
    y_min, x_min, y_max, x_max = bbox
    area = (y_max - y_min + 1) * (x_max - x_min + 1)
    return np.log(area)

def draw_truncate_gaussian(heatmap, center, h_radius, w_radius, k=1):
    h, w = 2 * h_radius + 1, 2 * w_radius + 1
    sigma_x = w / 6
    sigma_y = h / 6
    gaussian = gaussian2D((h, w), sigma_x=sigma_x, sigma_y=sigma_y)

    y, x = int(center[0]), int(center[1])

    height, width = heatmap.shape[0:2]

    left, right = min(x, w_radius), min(width - x, w_radius + 1)
    top, bottom = min(y, h_radius), min(height - y, h_radius + 1)

    masked_heatmap = heatmap[y - top : y + bottom, x - left : x + right]
    masked_gaussian = gaussian[h_radius - top : h_radius + bottom, w_radius - left : w_radius + right]
    if min(masked_gaussian.shape) > 0 and min(masked_heatmap.shape) > 0:
        np.maximum(masked_heatmap, masked_gaussian * k, out=masked_heatmap)
    return heatmap

def radius_ttf(bbox, h, w):
    alpha = 0.54
    h_radiuses_alpha = int(h / 2.0 * alpha)
    w_radiuses_alpha = int(w / 2.0 * alpha)
    return max(0, h_radiuses_alpha), max(0, w_radiuses_alpha)

def draw_heatmaps_ttf(shape, bboxes, labels):
    heat_map = np.zeros(shape, dtype=np.float32)
    box_target = np.ones((shape[0], shape[1], 4), dtype=np.float32)
    reg_weight = np.zeros((shape[0], shape[1], 1), dtype=np.float32)
    box_target_offset = np.zeros((shape[0], shape[1], 4), dtype=np.float32)

    meshgrid = get_pred_wh((shape[0], shape[1]))

    areas = np.asarray([bbox_areas_log_np(np.asarray(bbox)) for bbox in bboxes])
    indices = np.argsort(-areas)
    bboxes_new = bboxes[indices]
    labels_new = labels[indices]

    for bbox, cls_id in zip(bboxes_new, labels_new):
        bbox = np.asarray(bbox)
        area = bbox_areas_log_np(bbox)
        fake_heatmap = np.zeros((shape[0], shape[1]))
        w, h = bbox[3] - bbox[1], bbox[2] - bbox[0]

        if h > 0 and w > 0:
            # compute heat map
            h_radius, w_radius = radius_ttf(bbox, h, w)
            ct = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2], dtype=np.float32)
            ct_int = ct.astype(np.int32)
            draw_truncate_gaussian(fake_heatmap, ct_int, h_radius, w_radius)
            heat_map[:, :, cls_id] = np.maximum(heat_map[:, :, cls_id], fake_heatmap)

            # computes indices where is the current heatmap
            box_target_inds = fake_heatmap > 0

            # compute bbox size for current heatmap of bbox
            box_target[box_target_inds, :] = bbox[:]

            # this is just for debug/test
            box_target_offset[box_target_inds, 0] = meshgrid[1, box_target_inds] - bbox[0]
            box_target_offset[box_target_inds, 1] = meshgrid[0, box_target_inds] - bbox[1]
            box_target_offset[box_target_inds, 2] = bbox[2] - meshgrid[1, box_target_inds]
            box_target_offset[box_target_inds, 3] = bbox[3] - meshgrid[0, box_target_inds]

            # compute weight map for current heatmap of bbox
            local_heatmap = fake_heatmap[box_target_inds]
            ct_div = local_heatmap.sum() 
            local_heatmap *= area
            reg_weight[box_target_inds, 0] = local_heatmap / ct_div

    return heat_map, box_target, reg_weight, box_target_offset


In [ ]:
for samples in _tfrecords:
    #if samples['image/id']== 477442:
    image = samples["image"]
    originalShape = tf.shape(image)[:2]
    classes = tf.cast(samples["objects"]["label"], dtype=tf.int32)
    bbox_gt = samples["objects"]["bbox"]
    break

# create heatmap
bbox_rescaled = bbox_gt * 320/4.0
heatmap_size = 320 // 4
numofclass = 80
heatmap_shape = [heatmap_size, heatmap_size, numofclass] #[batch h w class]

shape = heatmap_shape
bboxes = bbox_rescaled
labels = classes

heat_map = np.zeros(shape, dtype=np.float32)
box_target = np.zeros((shape[0], shape[1], 4), dtype=np.float32)
reg_weight = np.zeros((shape[0], shape[1], 1), dtype=np.float32)
box_target_offset = np.zeros((shape[0], shape[1], 4), dtype=np.float32)

meshgrid = get_pred_wh((shape[0], shape[1]))

areas = np.asarray([bbox_areas_log_np(np.asarray(bbox)) for bbox in bbox_rescaled])
indices = np.argsort(-areas) #큰 박스부터 타겟에 넣음
bboxes_new = np.take(bbox_rescaled, indices, axis=0) #gather
labels_new = np.take(labels, indices)

for_print_ctv = []
for_print_area = []
for bbox, cls_id in zip(bboxes_new, labels_new):
    bbox = np.asarray(bbox)
    area = bbox_areas_log_np(bbox)
    fake_heatmap = np.zeros((shape[0], shape[1]))
    w, h = bbox[3] - bbox[1], bbox[2] - bbox[0]

    if h > 0 and w > 0:
        # compute heat map
        h_radius, w_radius = radius_ttf(bbox, h, w)
        ct = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2], dtype=np.float32)
        ct_int = ct.astype(np.int32)
        draw_truncate_gaussian(fake_heatmap, ct_int, h_radius, w_radius)
        heat_map[:, :, cls_id] = np.maximum(heat_map[:, :, cls_id], fake_heatmap)

        # computes indices where is the current heatmap
        box_target_inds = fake_heatmap > 0

        # compute bbox size for current heatmap of bbox
        box_target[box_target_inds, :] = bbox[:]

        # this is just for debug/test
        box_target_offset[box_target_inds, 0] = meshgrid[1, box_target_inds] - bbox[0]
        box_target_offset[box_target_inds, 1] = meshgrid[0, box_target_inds] - bbox[1]
        box_target_offset[box_target_inds, 2] = bbox[2] - meshgrid[1, box_target_inds]
        box_target_offset[box_target_inds, 3] = bbox[3] - meshgrid[0, box_target_inds]

        # compute weight map for current heatmap of bbox
        local_heatmap = fake_heatmap[box_target_inds]
        ct_div = local_heatmap.sum()
        ct_div = np.floor(ct_div)
        for_print_ctv.append(ct_div)
        for_print_area.append(area)
        
        local_heatmap *= area
        reg_weight[box_target_inds, 0] = local_heatmap / ct_div

#return heat_map, box_target, reg_weight, box_target_offset

In [ ]:
import math

def gaussian_radius(det_size, min_overlap=0.7):
    height, width = det_size

    a1 = 1
    b1 = height + width
    c1 = width * height * (1 - min_overlap) / (1 + min_overlap)
    sq1 = np.sqrt(b1 ** 2 - 4 * a1 * c1)
    r1 = (b1 + sq1) / 2

    a2 = 4
    b2 = 2 * (height + width)
    c2 = (1 - min_overlap) * width * height
    sq2 = np.sqrt(b2 ** 2 - 4 * a2 * c2)
    r2 = (b2 + sq2) / 2

    a3 = 4 * min_overlap
    b3 = -2 * min_overlap * (height + width)
    c3 = (min_overlap - 1) * width * height
    sq3 = np.sqrt(b3 ** 2 - 4 * a3 * c3)
    r3 = (b3 + sq3) / 2
    return min(r1, r2, r3)


def gaussian2D(shape, sigma_x=1, sigma_y=1):
    m, n = [(ss - 1.0) / 2.0 for ss in shape]
    y, x = np.ogrid[-m : m + 1, -n : n + 1]

    # h = np.exp(-(x * x + y * y) / (2 * sigma * sigma))
    h = np.exp(-(x * x / (2 * sigma_x * sigma_x) + y * y / (2 * sigma_y * sigma_y)))
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    return h

def draw_umich_gaussian(heatmap, center, radius, k=1):
    diameter = 2 * radius + 1
    gaussian = gaussian2D((diameter, diameter), sigma_x=diameter / 6, sigma_y=diameter / 6)

    y, x = int(center[0]), int(center[1])

    height, width = heatmap.shape[0:2]

    left, right = min(x, radius), min(width - x, radius + 1)
    top, bottom = min(y, radius), min(height - y, radius + 1)

    masked_heatmap = heatmap[y - top : y + bottom, x - left : x + right]
    masked_gaussian = gaussian[radius - top : radius + bottom, radius - left : radius + right]
    if min(masked_gaussian.shape) > 0 and min(masked_heatmap.shape) > 0:  # TODO debug
        np.maximum(masked_heatmap, masked_gaussian * k, out=masked_heatmap)
    return heatmap

def draw_heatmap(shape, bboxes, labels):
    heat_map = np.zeros(shape, dtype=np.float32)
    for bbox, cls_id in zip(bboxes, labels):
        h, w = bbox[3] - bbox[1], bbox[2] - bbox[0]
        if h > 0 and w > 0:
            radius = gaussian_radius((math.ceil(h), math.ceil(w)))
            radius = max(0, int(radius))
            ct = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2], dtype=np.float32)
            ct_int = ct.astype(np.int32)
            draw_umich_gaussian(heat_map[:, :, cls_id], ct_int, radius)

    return heat_map

In [ ]:
modelName = "MobileNetV3_FPN_CenterNet"

model_dir = "checkpoints/"
modelPart = modelName.split("_")

with open(os.path.join("model", "0_Config", modelName+".json"), "r") as config_file:
    config = json.load(config_file)

config['modelName'] = modelName
config['training_config']['num_classes'] = 80


##########
ids = tf.argsort(classes)
classes = tf.gather(classes, ids)
bboxes2 = tf.gather(bbox_gt, ids)
##########

le = LabelEncoder(config=config)
target = le._encode_sample(bboxes2, classes)

hm  = target[..., :80]
box  = target[..., 80:84]
rW  = target[..., 84:85]

In [ ]:
#heat_map = draw_heatmap([80, 80, 80], bboxes_new, labels_new)

In [ ]:
colors = np.random.rand(80, 3)*255

original_img = image.numpy()
for bbox, cls in zip(bbox_gt, classes):
    y1, x1, y2, x2 = bbox
    x1 = int(x1*image.shape[1])
    x2 = int(x2*image.shape[1])
    y1 = int(y1*image.shape[0])
    y2 = int(y2*image.shape[0])

    _text = '{}'.format(int(cls.numpy()))
    cv2.putText(original_img, _text, (x1,y1+10), cv2.FONT_HERSHEY_COMPLEX, 0.5, colors[int(cls.numpy())], thickness=1, lineType=cv2.LINE_AA)
    cv2.rectangle(original_img, (x1, y1), (x2, y2), (colors[int(cls.numpy())]), 1)

plt.figure(figsize=(8,8))
plt.imshow(cv2.resize(original_img, [480, 480]))

#################################################
plt.figure(figsize=(8,8))
plt.imshow(tf.reduce_max(hm, -1).numpy(), cmap='viridis')

plt.figure(figsize=(8,8))
plt.imshow(tf.reduce_max(heat_map, -1), cmap='viridis')
#################################################
plt.figure(figsize=(8,8))
box_np = box.numpy()*80.0
plt.imshow(box_np/255.0)

plt.figure(figsize=(8,8))
plt.imshow(box_target/255.0)
#################################################3
plt.figure(figsize=(8,8))
rW_show = rW.numpy()
plt.imshow(rW_show)

plt.figure(figsize=(8,8))
plt.imshow(reg_weight)

In [ ]:
t1 = tf.math.abs(box_np - box_target)

plt.figure(figsize=(8,8))
plt.imshow(tf.reduce_max(t1, -1).numpy()/255.0)

plt.figure(figsize=(8,8))
plt.imshow(np.abs(rW_show - reg_weight))

print("TP_HeatMap: ", np.abs(hm.numpy() - heat_map).max())
print("TP_BoxTarget: ", np.abs(box_np - box_target).max())
print("TP_rW: ", np.abs(rW_show - reg_weight).max())